# **Import Packages**

In [449]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', 200)

# **Loading data**

In [450]:
df = pd.read_csv('clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [451]:
df.head(5)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,var_year_price_off_peak_var,var_year_price_peak_var,var_year_price_mid_peak_var,var_year_price_off_peak_fix,var_year_price_peak_fix,var_year_price_mid_peak_fix,var_year_price_off_peak,var_year_price_peak,var_year_price_mid_peak,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,0.000061,2.627605e-05,0.000440,1.102785,49.550703,22.022535,1.102846,4.955073e+01,22.022975,0.000131,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0.000005,6.089453e-04,0.000000,0.006465,0.000000,0.000000,0.006470,6.089453e-04,0.000000,0.000003,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0.000006,2.558511e-07,0.000000,0.007662,0.000000,0.000000,0.007668,2.558511e-07,0.000000,0.000004,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0.000005,0.000000e+00,0.000000,0.006465,0.000000,0.000000,0.006470,0.000000e+00,0.000000,0.000003,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0.000015,3.552481e-06,0.000003,0.005429,0.001954,0.000869,0.005444,1.957971e-03,0.000871,0.000011,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0


In [481]:
df.shape

(14606, 45)

# **Feature engineering**

# **Difference between off-peak prices in December and preceding January**

In [452]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [482]:
price_df.shape

(193002, 8)

In [453]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


# **Merging Datasets into One Dataset**

In [454]:
merged_df = pd.merge(df, price_df, on='id', how='inner')
final_merged_df = pd.merge(merged_df, diff, on='id', how='inner')


In [455]:
final_merged_df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,var_year_price_off_peak_var,var_year_price_peak_var,var_year_price_mid_peak_var,var_year_price_off_peak_fix,var_year_price_peak_fix,var_year_price_mid_peak_fix,var_year_price_off_peak,var_year_price_peak,var_year_price_mid_peak,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.0,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,0.000061,0.000026,0.00044,1.102785,49.550703,22.022535,1.102846,49.550729,22.022975,0.000131,0.000041,0.000908,2.086294,99.530517,44.235794,2.086425,99.530558,44.236702,1,2015-01-01,0.125976,0.103395,0.071536,40.565969,24.339581,16.226389,0.020057,3.700961
1,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.0,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,0.000061,0.000026,0.00044,1.102785,49.550703,22.022535,1.102846,49.550729,22.022975,0.000131,0.000041,0.000908,2.086294,99.530517,44.235794,2.086425,99.530558,44.236702,1,2015-02-01,0.125976,0.103395,0.071536,40.565969,24.339581,16.226389,0.020057,3.700961
2,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.0,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,0.000061,0.000026,0.00044,1.102785,49.550703,22.022535,1.102846,49.550729,22.022975,0.000131,0.000041,0.000908,2.086294,99.530517,44.235794,2.086425,99.530558,44.236702,1,2015-03-01,0.125976,0.103395,0.071536,40.565973,24.339578,16.226383,0.020057,3.700961


In [456]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175149 entries, 0 to 175148
Data columns (total 53 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   id                               175149 non-null  object        
 1   channel_sales                    175149 non-null  object        
 2   cons_12m                         175149 non-null  int64         
 3   cons_gas_12m                     175149 non-null  int64         
 4   cons_last_month                  175149 non-null  int64         
 5   date_activ                       175149 non-null  datetime64[ns]
 6   date_end                         175149 non-null  datetime64[ns]
 7   date_modif_prod                  175149 non-null  datetime64[ns]
 8   date_renewal                     175149 non-null  datetime64[ns]
 9   forecast_cons_12m                175149 non-null  float64       
 10  forecast_cons_year               175149 non-

# **Extracting Year, Month, and Day Components from Datetime Columns**

In [457]:
datetime_columns = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal', 'price_date']
for column in datetime_columns:
    final_merged_df[column + '_year'] = final_merged_df[column].dt.year
    final_merged_df[column + '_month'] = final_merged_df[column].dt.month
    final_merged_df[column + '_day'] = final_merged_df[column].dt.day

In [458]:
final_merged_df['day_difference'] = final_merged_df['date_end_day'] - final_merged_df['date_activ_day']
final_merged_df['month_difference'] = final_merged_df['date_end_month'] - final_merged_df['date_activ_month']
final_merged_df['year_difference'] = final_merged_df['date_end_year'] - final_merged_df['date_activ_year']

In [459]:
datetime_columns = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal', 'price_date']
for column in datetime_columns:
    final_merged_df[column + '_year'] = final_merged_df[column].dt.year
    final_merged_df[column + '_month'] = final_merged_df[column].dt.month
    final_merged_df[column + '_day'] = final_merged_df[column].dt.day

# **Dropping columns after creating new features from old features**

In [460]:
columns_to_merge = ['date_end_day','date_activ_day','date_end_month','date_activ_month','date_end_year','date_activ_year']

In [461]:
final_merged_df.drop(datetime_columns, axis=1, inplace=True)
final_merged_df.drop(columns_to_merge, axis=1, inplace=True)

# **Label Encoding to Categorical Features**

In [462]:
def label_encoder(df, columns_to_encode):
    le = LabelEncoder()
    for col in columns_to_encode:
        df[col] = le.fit_transform(df[col].astype(str))
    return df


In [463]:
columns_to_encode = ['channel_sales', 'origin_up','has_gas']
final_merged_df = label_encoder(final_merged_df, columns_to_encode)

In [464]:
print(final_merged_df.dtypes)

for col in final_merged_df.columns:
    if final_merged_df[col].dtype == 'object' or final_merged_df[col].nunique() < 10:
        print(f"{col}: {final_merged_df[col].nunique()} unique values")


id                                  object
channel_sales                        int64
cons_12m                             int64
cons_gas_12m                         int64
cons_last_month                      int64
forecast_cons_12m                  float64
forecast_cons_year                   int64
forecast_discount_energy           float64
forecast_meter_rent_12m            float64
forecast_price_energy_off_peak     float64
forecast_price_energy_peak         float64
forecast_price_pow_off_peak        float64
has_gas                              int64
imp_cons                           float64
margin_gross_pow_ele               float64
margin_net_pow_ele                 float64
nb_prod_act                          int64
net_margin                         float64
num_years_antig                      int64
origin_up                            int64
pow_max                            float64
var_year_price_off_peak_var        float64
var_year_price_peak_var            float64
var_year_pr

# **Merging Columns into one column**

In [465]:
final_merged_df['combined_category'] = final_merged_df['channel_sales'].astype(str) + "_" + final_merged_df['has_gas'].astype(str)

In [466]:
df['total_forecast_price'] = df['forecast_price_energy_off_peak'] + df['forecast_price_energy_peak'] + df['forecast_price_pow_off_peak']


In [467]:
columns_to_drop = ['channel_sales','has_gas','forecast_price_energy_off_peak','forecast_price_energy_peak','forecast_price_pow_off_peak']

In [468]:
final_merged_df.drop(columns_to_drop, axis=1, inplace=True)

# **Checking Missing Values**

In [469]:
final_merged_df.isnull().sum()

id                                 0
cons_12m                           0
cons_gas_12m                       0
cons_last_month                    0
forecast_cons_12m                  0
forecast_cons_year                 0
forecast_discount_energy           0
forecast_meter_rent_12m            0
imp_cons                           0
margin_gross_pow_ele               0
margin_net_pow_ele                 0
nb_prod_act                        0
net_margin                         0
num_years_antig                    0
origin_up                          0
pow_max                            0
var_year_price_off_peak_var        0
var_year_price_peak_var            0
var_year_price_mid_peak_var        0
var_year_price_off_peak_fix        0
var_year_price_peak_fix            0
var_year_price_mid_peak_fix        0
var_year_price_off_peak            0
var_year_price_peak                0
var_year_price_mid_peak            0
var_6m_price_off_peak_var          0
var_6m_price_peak_var              0
v

# **Splitting Data Into X and Y to do co-relation and to delete features which is not important**

In [470]:
X = final_merged_df.drop(['id','churn'], axis=True)
y = final_merged_df['churn']

In [471]:
numeric_df = X.select_dtypes(include=[np.number])

In [472]:
corr_matrix = numeric_df.corr()

In [473]:
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

In [474]:
threshold = 0.9

In [475]:
columns_to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]


In [476]:
X_reduced = numeric_df.drop(columns=columns_to_drop, axis=1)


# **Here Reduced Data**

In [479]:
X_reduced.head()

,cons_12m,cons_gas_12m,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,margin_gross_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,var_year_price_off_peak_var,var_year_price_peak_var,var_year_price_mid_peak_var,var_year_price_off_peak_fix,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,date_modif_prod_year,date_modif_prod_month,date_modif_prod_day,date_renewal_year,date_renewal_month,date_renewal_day,price_date_year,price_date_month,price_date_day,day_difference,month_difference
0,0,54946,0.0,0,0.0,1.78,25.44,2,678.99,3,4,43.648,0.000061,0.000026,0.00044,1.102785,0.000131,0.000041,0.000908,0.125976,0.103395,0.071536,40.565969,0.020057,3.700961,2015,11,1,2015,6,23,2015,1,1,0,0
1,0,54946,0.0,0,0.0,1.78,25.44,2,678.99,3,4,43.648,0.000061,0.000026,0.00044,1.102785,0.000131,0.000041,0.000908,0.125976,0.103395,0.071536,40.565969,0.020057,3.700961,2015,11,1,2015,6,23,2015,2,1,0,0
2,0,54946,0.0,0,0.0,1.78,25.44,2,678.99,3,4,43.648,0.000061,0.000026,0.00044,1.102785,0.000131,0.000041,0.000908,0.125976,0.103395,0.071536,40.565973,0.020057,3.700961,2015,11,1,2015,6,23,2015,3,1,0,0
3,0,54946,0.0,0,0.0,1.78,25.44,2,678.99,3,4,43.648,0.000061,0.000026,0.00044,1.102785,0.000131,0.000041,0.000908,0.125976,0.103395,0.071536,40.565973,0.020057,3.700961,2015,11,1,2015,6,23,2015,4,1,0,0
4,0,54946,0.0,0,0.0,1.78,25.44,2,678.99,3,4,43.648,0.000061,0.000026,0.00044,1.102785,0.000131,0.000041,0.000908,0.125976,0.103395,0.071536,40.565973,0.020057,3.700961,2015,11,1,2015,6,23,2015,5,1,0,0


In [478]:
X_reduced.shape

(175149, 36)